# Calculating SOAP descriptors for zeolites

In this notebook, we will calculate the SOAP descriptors for IZA zeolites using the package `dscribe`.

In [1]:
import os
import warnings
import pandas as pd
import numpy as np
from ase.io import read
from dscribe.descriptors import SOAP

warnings.filterwarnings("ignore")

## Calculation of the SOAP descriptors

In [2]:
species = ["Si", "O"]
r_cut = 6.0
n_max = 8
l_max = 6

# Setting up the SOAP descriptor
soap = SOAP(
    species=species,
    periodic=True,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner",
)

In [3]:
path = os.path.abspath('../dsets/iza_experimental')
files = sorted([f for f in os.listdir(path) if f.endswith(".cif")])

zeolites = {}
for f in files:
    iza = f.strip(".cif")
    atoms = read(os.path.join(path, f))
    zeolites[iza] = atoms

In [4]:
%%time

soap_vecs = np.array([
    soap.create(atoms) for atoms in zeolites.values()
])

dotprod = (soap_vecs @ soap_vecs.T)
norms = np.diag(dotprod)
kernel = dotprod / np.sqrt(norms.reshape(-1, 1) * norms.reshape(1, -1))
dm = np.sqrt(2 - 2 * kernel)

CPU times: user 6.09 s, sys: 1.9 s, total: 7.99 s
Wall time: 5.9 s


In [5]:
dm = pd.DataFrame(dm, index=zeolites.keys(), columns=zeolites.keys())